<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Импорт библиотек

In [ ]:
# pip install nltk # доустанавливаем необходимые библиотеки
# pip install pywsd
# pip install transformers

In [2]:
import pandas as pd
import torch
import transformers 
import transformers as ppb # pytorch transformers

import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from tqdm import notebook

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

from sklearn.metrics import f1_score


from urllib.parse import urlencode # для задания адреса таблицы в я.диске
import requests #получить финальный url я.диска

import pandas as pd
import nltk

import re 
from pywsd.utils import lemmatize_sentence #лемматизатор
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer 


Warming up PyWSD (takes ~10 secs)... took 5.001153945922852 secs.


### Подготовка констант

In [3]:
RANDOM_STATE = 100
max_len = 512

### Знакомство с данными

In [2]:
try:
    df = pd.read_csv('toxic_comments.csv')
except:
    df = pd.read_csv(r'C:\Users\maxpe\Downloads\Practicum\Projects\datasets\toxic_comments.csv')
display(df.shape);

(159292, 3)

In [ ]:
df.info()

In [4]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


Датасет не содержит пустых строк, состоит из 3 столбцов, один из которых будет преобразован в признаки (`text`), а `toxic` - целевой признак. Столбец `Unnamed` можем удалить. 

In [3]:
df = df.drop(['Unnamed: 0'], axis=1)

## BERT

### Подготовка данных для BERT

#### Загрузка предобученных модели и токенизатора.

In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights);

Проверим любой рандомный текст.

In [ ]:
tokenizer.encode('You, sir, are my hero. Any chance you remember..', add_special_tokens=True)

Токенизатор отрабатывает корректно. Применим его предварительно к датасету.

#### Преобразуем текст

In [ ]:
pre_tokenized = df['text'].apply(
  lambda x: tokenizer.encode(x, add_special_tokens=True)) 

In [ ]:
pre_tokenized

Как видим из сообщения к `pre_tokenized`, имеем ограничение в 512 токенов в строке. Создадим такое ограничение.

#### Фильтруем данные

Собираем датасет из токенизированных данных - столбец `text`, целевого признака и длин списков токенов.

In [ ]:
df_temp = pd.DataFrame(pre_tokenized)
df_temp['toxic'] = df['toxic']
len_list = []
for i in df_temp['text']:
    len_list.append(len(i))
df_temp['len'] = len_list
df_temp

Удаляем строки с превышающими лимит токенами. таких строк порядка 1.8%

In [ ]:
df_temp = df_temp.loc[df_temp['len'] < 513].drop(['len'], axis=1)

Собираем данные по индексу: слева - тексты из исходного датасета, справа - целевой признак из нового, отфильтрованного датасета. Так как `join` имеет значение `inner`, то остаются только те строки, в которых совпадает индекс, то есть прошедшие фильтрацию.

In [ ]:
df_left = df['text']
df_right = df_temp['toxic']
result = pd.concat([df_left, df_right], axis=1, join="inner")
result

В случае преобразования в эмбенддинги внутри ноутбука, код ниже - указывает количество семплов для будущего датасета, где `q_sample = len(result)` - это полный датасет.

Ниже приведен пример расчета для очень маленькой выборки - взяли 100 строк, чтобы показать работоспособность кода. 

Полный преобразованный датасет будет использоваться уже на этапе разделения признаков и целевого признака.

In [ ]:
q_sample = 100 # нужное число для датасета, вплоть до q_sample = len(result)
result = result.sample(q_sample).reset_index(drop=True)

#### Преобразуем текст предобработанных данных

Токенизируем отфильтрованную выборку.

In [ ]:
tokenized = result['text'].apply(
  lambda x: tokenizer.encode(x, add_special_tokens=True)) 

Предупреждения больше нет. Максимальная длина токена `max_len` равна ограничению - 512. Вынесли это значения в константы.

При исполнении преобразования внутри ноутбука, `max_len` расчитывается ниже. Так как мы случайным образом отберем какое-то количество строк, может случиться, что строки будут короче константной `max_len = 512`, это нужно учитывать

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
max_len

#### Преобразование векторов

Паддингом добавляем к каждому вектору нули в конец, так, чтобы длина каждого вектора была равна длине `max_len`.

Маска тоже применяется к каждому вектору - значению, отличному от нуля, присваивается 1, нули остаются нулями.

In [ ]:
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

Заранее рассчитанные данные:
- В Яндекс.Диске лежит датасет, содержащий 78% данных (ограничение по размеру файла), при батче равном 200. Он будет прочитан позже.

Преобразование эмбеддингов было произведено в Google Colab, с использованием GPU.

#### Преобразование в эмбеддинги

In [ ]:
batch_size = 50 # задаем размер батча (кратен q_sample, если уменьшаем q_sample - уменьшаем и батч)
embeddings = [] # список эмбеддингов
for i in notebook.tqdm(range(padded.shape[0] // batch_size)): # цикл по батчам, отображать прогресс будет функция notebook()
        batch = torch.IntTensor(padded[batch_size*i:batch_size*(i+1)]) # преобразуем данные в формат тензоров
        attention_mask_batch = torch.IntTensor(attention_mask[batch_size*i:batch_size*(i+1)]) # преобразуем маску
        
        with torch.no_grad(): # для ускорения вычисления функцией no_grad() в библиотеке torch укажем, что градиенты не нужны
            batch_embeddings = model(batch, attention_mask=attention_mask_batch) # чтобы получить эмбеддинги для батча, передадим модели данные и маску
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy()) # преобразуем элементы методом numpy() к типу numpy.array

df_ed = pd.DataFrame(np.concatenate(embeddings)) # соберём все эмбеддинги в матрицу признаков вызовом функции concatenate()
df_ed['toxic'] = result['toxic'] # добавляем целевой признак
df_ed

Код выше, показывает механизм преобразования эмбеддингов на маленькой выборке. То же самое было расчитано на всем датасете. Прочитаем его.

#### Выгрузка датасета

In [ ]:
# df_ed.to_csv('df_ed_20k.csv', index=False)

Выгружаем датасет после этапа преобразования в эмбеддинги, для того, чтобы не пересчитывать их в следующий сеанс или на другом устройстве. Выгрузка датасета закомментирована, она не должна больше исполняться.

#### Загрузка датасета

Рассчитанные эмбендинги в Colab я сохранил в Яндекс.Диске. Здесь сохранен неполный датасет, порядка 78% от исходного. Прочитаем `csv-файл`.

In [ ]:
# используем api 
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' 
public_key = 'https://disk.yandex.ru/d/KLot7E--emZcKg' 
 
# получаем url 
final_url = base_url + urlencode(dict(public_key=public_key)) 
response = requests.get(final_url) 
download_url = response.json()['href'] 
 
# загружаем файл в df 
download_response = requests.get(download_url)
df_ed_78 = pd.read_csv(download_url, sep=',')
df_ed_78

In [ ]:
df_ed_78 = df_ed_78.astype('float32')
df_ed_78['toxic'] = df_ed_78['toxic'].astype('int')
df_ed_78.info()

### Обучение для BERT

#### Ограничение датасета

Так как датасет получается тяжелым, чтобы была возможность на его данных провести обучение, их нужно немного разделить. Сделаем это с помощью `train_test_split`, чтобы сохранить соотношение в целевом признаке.

Соотношение 0.1 - это 10% на второй сет в сплите, то есть порядка 12 тысяч строк будет при таком соотношении во втором датасете.

In [ ]:
out, df_ed = train_test_split(df_ed_78, test_size=0.1, random_state=RANDOM_STATE, stratify= df_ed_78['toxic']) 

In [ ]:
df_ed

#### Разбиение данных

In [ ]:
features_train, features_test, target_train, target_test = train_test_split( #создаем 4 датасета, два признаков (тест+валидация) и два целевых, 
    df_ed.drop(columns='toxic'), #для датасетов признаков удаляем целевой
    df_ed['toxic'], #для целевого оставляем только целевой
    test_size=0.2, #с соотношением 75/25
    random_state=RANDOM_STATE, #с заданной опорой для рандома 
    stratify= df_ed['toxic']) #с заданной стратификацией по целевому признаку

#### Модель логистической регрессии

In [ ]:
%%time

log_reg = LogisticRegression(solver='lbfgs', max_iter=1000, class_weight= 'balanced') #максимальное количество итераций 1000, 
log_reg_scores = cross_val_score(log_reg, features_train, target_train.values, scoring='f1', cv=10)
log_reg_f1 = log_reg_scores.mean()

print("Среднее значение метрики F1 для модели логистической регрессии "\
    "с использованием кросс-валидации:", log_reg_f1)

#### Модель опорных векторов

In [ ]:
%%time

svclassifier = SVC(kernel='linear') 
svclassifier_scores = cross_val_score(svclassifier, features_train, target_train.values, scoring='f1', cv=10)
svclassifier_f1 = svclassifier_scores.mean()

print("Среднее значение метрики F1 для модели SVC "\
    "с использованием кросс-валидации:", svclassifier_f1)

#### Модель случайного леса

In [ ]:
%%time

forest = RandomForestClassifier(random_state=RANDOM_STATE, class_weight= 'balanced') #модель случайного леса
parameters = {'n_estimators': range (1, 300), 'max_depth': range (1, 50)} #перебор гиперпараметров
#применение метода гридсёрч со встроенной кросс-валидацией к модели леса с перебором указанных параметров

randomized_forest = RandomizedSearchCV(forest, n_iter=20, param_distributions= parameters, scoring='f1', n_jobs= -3, cv=5)
#обучение модели
randomized_forest.fit(features_train, target_train.values)

#лучшее значение после перебора параметров 
best_forest = abs(randomized_forest.best_score_)

print("Лучшие параметры для модели случайного леса с "\
    "использованием кросс-валидации:", randomized_forest.best_params_)
print("Наибольшее значение метрики F1 для модели случайного леса "\
    "при лучших гиперпараметрах с использованием кросс-валидации:", best_forest)

#### LightGBM

In [ ]:
%%time

lgbm = lgb.LGBMClassifier() #
parameters = {'n_estimators': range (1, 1000), 'max_depth': range (1, 100), 'learning_rate': (0.05, 0.5, 0.05)}
#применение метода гридсёрч со встроенной кросс-валидацией

rand_lgbm = RandomizedSearchCV(lgbm, n_iter=20, param_distributions= parameters, scoring='f1', n_jobs= -3, cv=5)
#обучение модели
rand_lgbm.fit(features_train, target_train.values)

#лучшее значение после перебора параметров 
best_lgbm = rand_lgbm.best_score_

print("Лучшие параметры для модели случайного леса с "\
    "использованием кросс-валидации:", rand_lgbm.best_params_)
print("Наибольшее значение метрики F1 для модели LGBMClassifier "\
    "при лучших гиперпараметрах с использованием кросс-валидации:", best_lgbm)

### Выводы по методу BERT

Мы получили довольно неплохие показатели метрики F1, на модели опорных векторов значением 0.68. Чего, в свою очередь недостаточно, для выполнения этой задачи.

## TF-IDF

### Лемматизация

In [212]:
dft = pd.read_csv(r'C:\Users\maxpe\Downloads\Practicum\Projects\datasets\toxic_comments.csv')
dft = dft.drop(['Unnamed: 0'], axis=1)

In [3]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maxpe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maxpe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\maxpe\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [214]:
pd.options.mode.chained_assignment = None

In [215]:
def clear_text(text):

    def limit(text):
        lim = re.sub(r'\b\w{50,10000}\b', ' ', text)
        lim1 = " ".join(lim.split())
        
        return lim1

    clear = re.sub(r'[^a-zA-Z ]', ' ', limit(text))
    clear1 = " ".join(clear.split())
    
    return clear1

In [216]:
def lemmatize_text(text):
    lemma = lemmatize_sentence(text)
    return " ".join(lemma)

In [217]:
dft['lemma'] = dft['text'].apply(clear_text)

In [4]:
dft = pd.read_csv(r'C:\Users\maxpe\Downloads\Practicum\Projects\project13_BERT\df_lemma.csv')

In [5]:
# dft['lemma'] = dft['lemma'].apply(lemmatize_text)

In [6]:
# dft.to_csv('df_lemma.csv', index=False)

In [7]:
df_lem = dft
df_lem.head()

,text,toxic,lemma
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


In [8]:
features_train_lem, features_test_lem, target_train_lem, target_test_lem = train_test_split( #создаем 4 датасета, два признаков (тест+валидация) и два целевых, 
    df_lem.drop(columns='toxic'), #для датасетов признаков удаляем целевой
    df_lem['toxic'], #для целевого оставляем только целевой
    test_size=0.2, #с соотношением 
    random_state=RANDOM_STATE, #с заданной опорой для рандома 
    stratify= df_lem['toxic']) #с заданной стратификацией по целевому признаку

In [9]:
nltk.download('stopwords') 
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maxpe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
corpus_train = features_train_lem['lemma'].values.astype('U')
count_tf_idf_train = TfidfVectorizer(stop_words=stop_words)
tf_idf_train = count_tf_idf_train.fit_transform(corpus_train) 
tf_idf_train.shape

(127433, 134054)

In [11]:
corpus_test = features_test_lem['lemma'].values.astype('U')
tf_idf_test = count_tf_idf_train.transform(corpus_test)
tf_idf_test

<31859x134054 sparse matrix of type '<class 'numpy.float64'>'
	with 824502 stored elements in Compressed Sparse Row format>

#### Модель логистической регрессии

In [28]:
%%time

log_reg_lem = LogisticRegression(solver='lbfgs', max_iter=1000, class_weight= 'balanced') #максимальное количество итераций 1000, 
log_reg_scores_lem = cross_val_score(log_reg_lem, tf_idf_train, target_train_lem.values, scoring='f1', cv=10)
log_reg_f1_lem = log_reg_scores_lem.mean()

print("Среднее значение метрики F1 для модели логистической регрессии "\
    "с использованием кросс-валидации:", log_reg_f1_lem)

Среднее значение метрики F1 для модели логистической регрессии с использованием кросс-валидации: 0.7443337162368571
CPU times: total: 7.31 s
Wall time: 30.7 s


#### Модель опорных векторов

In [29]:
%%time

svclassifier_lem = SVC(kernel='linear', class_weight= 'balanced') 
svclassifier_scores_lem = cross_val_score(svclassifier_lem, tf_idf_train, target_train_lem.values, scoring='f1', cv=2)
svclassifier_f1_lem = svclassifier_scores_lem.mean()

print("Среднее значение метрики F1 для модели SVC "\
    "с использованием кросс-валидации:", svclassifier_f1_lem)

KeyboardInterrupt: 

#### Модель случайного леса

In [30]:
%%time

forest_lem = RandomForestClassifier(random_state=RANDOM_STATE, class_weight= 'balanced') #модель случайного леса
parameters = {'n_estimators': range (1, 1000, 50), 'max_depth': range (1, 100, 5)} #перебор гиперпараметров
#применение метода гридсёрч со встроенной кросс-валидацией к модели леса с перебором указанных параметров

randomized_forest_lem = RandomizedSearchCV(forest_lem, n_iter=15, param_distributions= parameters, scoring='f1', n_jobs= -2, cv=2)
#обучение модели
randomized_forest_lem.fit(tf_idf_train, target_train_lem.values)

#лучшее значение после перебора параметров 
best_forest_lem = abs(randomized_forest_lem.best_score_)

print("Лучшие параметры для модели случайного леса с "\
    "использованием кросс-валидации:", randomized_forest_lem.best_params_)
print("Наибольшее значение метрики F1 для модели случайного леса "\
    "при лучших гиперпараметрах с использованием кросс-валидации:", best_forest_lem)

#### LightGBM

In [12]:
%%time

lgbm_lem = lgb.LGBMClassifier() #
parameters = {'n_estimators': range (1, 500, 10), 'max_depth': range (1, 100, 5), 'learning_rate': (0.1, 0.8, 0.1)}
#применение метода гридсёрч со встроенной кросс-валидацией

rand_lgbm_lem = RandomizedSearchCV(lgbm_lem, n_iter=20, param_distributions= parameters, scoring='f1', n_jobs= -2, cv=3)
#обучение модели
rand_lgbm_lem.fit(tf_idf_train, target_train_lem.values)

#лучшее значение после перебора параметров 
best_lgbm_lem = rand_lgbm_lem.best_score_

print("Лучшие параметры для модели случайного леса с "\
    "использованием кросс-валидации:", rand_lgbm_lem.best_params_)
print("Наибольшее значение метрики F1 для модели LGBMClassifier "\
    "при лучших гиперпараметрах с использованием кросс-валидации:", best_lgbm_lem)

Лучшие параметры для модели случайного леса с использованием кросс-валидации: {'n_estimators': 421, 'max_depth': 91, 'learning_rate': 0.1}
Наибольшее значение метрики F1 для модели LGBMClassifier при лучших гиперпараметрах с использованием кросс-валидации: 0.7691240046453833
CPU times: total: 7min 15s
Wall time: 7min 30s


### Проверяем лучшую модель

In [14]:
%%time

lgbm_lem_1 = lgb.LGBMClassifier(
    n_estimators=rand_lgbm_lem.best_params_['n_estimators'], 
    max_depth=rand_lgbm_lem.best_params_['max_depth'], 
    learning_rate=rand_lgbm_lem.best_params_['learning_rate'])
lgbm_lem_1.fit(tf_idf_train, target_train_lem.values)

predictions_lgbm_lem_1 = lgbm_lem_1.predict(tf_idf_test)

lgbm_lem_f1 = f1_score(target_test_lem, predictions_lgbm_lem_1)
print("Наименьшее значение метрики RMSE на тестовых данных", lgbm_lem_f1)

Наименьшее значение метрики RMSE на тестовых данных 0.7736796536796536
CPU times: total: 7min 4s
Wall time: 31.4 s


## Выводы

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны